In [295]:
import pandas as pd
import numpy as np
import re
import json
import os
import shutil


# Choosing Clusters
We accomplished clustering among first original dataset and received 100 clusters. Then we put visualisation of those clusters to ChatGPT and asked to name each cluster as if it was taxonomy name (details in Trello). Here we  
- want to extract those cluster names and choose relevant  
- pit them to MVP

## Retrieve and orginize ChatGPT taxonomy


In [296]:
file_name = 'cluster no = 100  random_state=42/taxonomy_clustering100.txt'

with open(file_name, 'r') as f:
    chatgpt_taxonomy  = f.read().splitlines()

chatgpt_taxonomy = [line for line in chatgpt_taxonomy if line != '']
cluster_info = {}

cluster_num = '0'
is_description = False
for line in chatgpt_taxonomy:
    if line == '':
        continue
    if r'Cluster_' in line:
        cluster_num = int(re.search(r'Cluster_(\d{0,3})', line).group(1))
        cluster_info[cluster_num] = {}
        continue
    if 'Cluster Name' in line:
        name = line[re.search('Cluster Name: ', line).span()[1]:]
        cluster_info[cluster_num]['name'] = name.strip()
    if 'Key Words:' in line:
        key_words = line[re.search('Key Words: ', line).span()[1]:].split(', ')
        cluster_info[cluster_num]['keywords'] = key_words
        continue
    if 'Detailed Description:' == line:
        is_description = True
        continue
    if is_description:
        is_description = False
        cluster_info[cluster_num]['description'] = line.strip()
        continue
    if 'Detailed Description:' in line:
        description = line[re.search('Detailed Description:', line).span()[1]:].strip()
        cluster_info[cluster_num]['description'] = description
    

In [297]:
print('Amount of clusters retrieved:', len(cluster_info))
cluster_info[98]

Amount of clusters retrieved: 100


{'name': 'Architectural Ceilings and Vaults',
 'keywords': ['ceilings',
  'vaults',
  'stained glass',
  'arches',
  'architectural details',
  'religious architecture',
  'geometric patterns'],
 'description': 'This cluster highlights intricate architectural ceilings and vault designs. It includes a variety of styles such as Gothic ribbed vaults, stained glass windows, and ornate decorative elements. The images showcase the intersection of structural engineering and artistic embellishment, often found in religious buildings like cathedrals, churches, and historical monuments. The collection emphasizes the beauty of ceiling craftsmanship, combining geometric precision with artistic flair.'}

In [298]:
with open('cluster no = 100  random_state=42/clusters_info.json', 'w') as f:
    json.dump(cluster_info, f, indent=4)

In [299]:
names, descriptions = [], []

# # Save names as list of tuples (cluster_num, name)
# for i in cluster_info:
#     names.append((i, cluster_info[i]['name']))
# # names = sorted(names, key=lambda x: x[1])

# Save names as list
for i in cluster_info:
    names.append(cluster_info[i]['name'])
    descriptions.append(cluster_info[i]['description'])
    
tockens = set([])
for name in names:
    tockens.update(set(name.split(' ')))

# names[:5], descriptions[:5]

In [300]:
## Attempt to choose tockens that are relevant to architecture 

# relevant_tockens = ['Architecture', 'Church', 'Building', 'Buildings', 
#                     'Construction', 'Exteriors', 'Facades', 'Gothic', 'Landscape', 'Landscapes',
#                     'Monuments', 'Mountainous', 'Nature', 'Residential', 'Rural',
#                     'Settlements', 'Site', 'Sites', 'Spaces', 'Studios', 'Transport',
#                     'Urban',]

# for i in cluster_info:
#     if any(relevant_tocken in cluster_info[i]['name'] for relevant_tocken in relevant_tockens):
#         print(i, cluster_info[i]['name'])

In [301]:
# with open("cluster no = 100  random_state=42/cluster_names.txt", "w", encoding="utf-8") as file:
#     for name in names:
#         file.write(name + '\n')


## Ask GPT to choose relevant

ChatGPT prompt:  
I did a clustering on archive images. Now I have names for those clusters. I need to extract all clusters (based on names) which relate to picutres of outside buildings
..

In [302]:
## Respond 1
# with open('cluster no = 100  random_state=42/gptresponse_relevant_outside_1.txt', 'r') as f:
#     relevant_names = f.read().splitlines()

# relevant_names = [name.split(' - ')[0] for name in relevant_names if 'Yes' in name]

# relevant_outdoors = []
# for i in cluster_info:
#     if cluster_info[i]['name'] in relevant_names:
#         relevant_outdoors.append(i)


In [303]:
# ## Respond 2
# with open('cluster no = 100  random_state=42/gptresponse_relevant_outside_2.txt', 'r') as f:
#     relevant_names = f.read().splitlines()

# relevant_names = [line.split(' - ')[0] for line in relevant_names if line != '' and 'Yes' in line]

# # relevant_outdoors = []
# # for i in cluster_info:
# #     if cluster_info[i]['name'] in relevant_names:
# #         relevant_outdoors.append(i)
# relevant_names

For manualy method maybe add agrement on 3 people

In [304]:
## Manualy
relevant_outdoors = [0, 1, 4, 5, 12, 22, 33, 40, 49, 65, 66, 74, 83, 99]
relevant_indoor = [3, 19, 61, 62, 64,67,84,]
relevant_details = [35, 51, 68, 2]
relevant_people = [11, 14, 24, 44, 47, 48, 50, 56, 57, 58,59, 63, 77,82, 89,95]
# Not sure about 52

In [305]:
# Evaluate relevance of chosen clusters by keywords (manually) 
for i in cluster_info:
    if i in relevant_outdoors:
        print(i, '\t', cluster_info[i]['name'])
        # print(cluster_info[i]['description'])
        print(cluster_info[i]['keywords'])
        print()

0 	 Architectural Archways
['arches', 'domes', 'interiors', 'bridges', 'structures']

1 	 Historical Landscape Prints
['landscapes', 'black-and-white', 'mountains', 'architecture', 'nature']

4 	 Black-and-White Landscapes
['mountains', 'towns', 'rivers', 'architecture', 'valleys']

5 	 Historical Church Facades
['cathedrals', 'spires', 'Gothic', 'architecture', 'towers']

12 	 Urban Landscapes
['buildings', 'architecture', 'cityscapes', 'aerial', 'landmarks']

22 	 Architectural Facades
['buildings', 'facades', 'architecture', 'houses', 'styles']

33 	 Urban Architectural Styles
['buildings', 'streets', 'facades', 'urban', 'historical']

40 	 Historical Buildings and Landscapes
['buildings', 'landscapes', 'ruins', 'villages', 'architecture']

49 	 Architectural Facades
['buildings', 'facades', 'black-and-white', 'windows', 'roofs']

65 	 Construction Site Documentation
['construction', 'buildings', 'scaffolding', 'architecture', 'cranes']

66 	 Residential Building Exteriors
['houses'

In [306]:
final_clusters = relevant_outdoors

EVALUATE RESULT. Seems bullshit
Specify system prompt  

Maybe **manualy** is better
Need:
- landscape with nature
- exterior and city
- interior
- people in interior

Or maybe choose keywords with code

## Concatenate df_MVP and clustering label

In [307]:
df_cluster = pd.read_csv('cluster no = 100  random_state=42/file_clusters_100.csv', sep=';')
df_cluster.rename(columns={'File Name': 'file_name', 'Cluster': 'cluster'}, inplace=True)
df_cluster['file_name'] = df_cluster['file_name'].apply(lambda x: x.split('.')[0])

df_mvp = pd.read_csv('datasets/dataset_MVP.csv', sep=';')

df_mvp.shape, df_cluster.shape

((37185, 5), (36981, 2))

In [308]:
df = df_mvp.merge(df_cluster, on='file_name', how='left')
df.dropna(subset=['cluster'], inplace=True)
df['cluster'] = df['cluster'].astype(int)
df['file_name'] = df['file_name'].apply(lambda x: x+'.jpg')
print(df.shape)
df.head()

(36981, 6)


,link,id_objects,labels,file_name,id,cluster
0,http://www.bildindex.de/bilder/d/ae00001b05,['9077'],[1617],scn-ae00001b05.jpg,ae00001b05,85
1,http://www.bildindex.de/bilder/d/ae00001b06,['9077'],[1617],scn-ae00001b06.jpg,ae00001b06,85
2,http://www.bildindex.de/bilder/d/ae00001b07,['9077'],[1617],scn-ae00001b07.jpg,ae00001b07,85
3,http://www.bildindex.de/bilder/d/ae00001b08,['9077'],[1617],scn-ae00001b08.jpg,ae00001b08,96
4,http://www.bildindex.de/bilder/d/ae00001b10,['3899'],[3257],scn-ae00001b10.jpg,ae00001b10,1


In [ ]:
df.to_csv('datasets/dataset_MVP_clustered.csv', sep=';', index=False)

## Choose relevant outdoor images

In [309]:
chosen_images_out = df[df.cluster.isin(final_clusters)].file_name.tolist()
print('Amount images to work with:', len(chosen_images_out))

if not os.path.exists('images/images_outdoor_compressed'):
    os.makedirs('images/images_outdoor_compressed')

for image in chosen_images_out:
    shutil.copy('images/images_compressed/' + image, 'images/images_outdoor_compressed/' + image)

Amount images to work with: 6796


In [314]:
# # Save as csv
df[df.cluster.isin(final_clusters)].to_csv('datasets/outdoor_images.csv', sep=';', index=False)